# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm#MID), thank goodness we can search for these things.

## Preparation: Knowing your tags

These questions are the same for every data set, and might not work exactly for yours.

**Search for every operator with 'dirt' in their name, including abandoned mines.**

### What is the tag and class name for every row of data?

In [ ]:
# tag name of every row: <tr>, class: none

### What is the tag and class name for every mine operator's name?

In [ ]:
# tag name of mine operator's name: <td>, class: none

### What is the tag and class name for every mine's name?

In [ ]:
# tag name of mine's name: <td>, class: none

### What is the tag and class name for every mine operator's name?

In [ ]:
# tag name of mine operator's name: <td>, class: none

### What is the tag and class name for every mine operator's name?

In [ ]:
# tag name of mine operator's name: <td>, class: none

## Being lazy

If you only needed these results, what would you do instead of scraping them?

## Setup: Import what you'll need to scrape the page

Use `requests`, not `urllib`.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# We have to send data with the form request (see below)

In [3]:
data = {
    'OperSearch':'dirt',
    'Abandoned':'No',
    'MineName':'',
    'StateSearch':'None',
    'CM':'All',
    'x':'0',
    'y':'0',
    'MC':'Opersearch'
}

url = 'https://arlweb.msha.gov/drs/ASP/OprNameStatesearch.asp'
response = requests.post(url, data=data)

In [4]:
doc = BeautifulSoup(response.text, 'html.parser')

## Try to scrape the page

To test if you requested the page correctly, save the BeautifulSoup document as `doc` and run the code `doc.find_all('tr')[-1].text` to get the text of the last `<tr>` element.

- If the result starts with **Total Number of Mines Found**, you were successful.

In [5]:
doc.find_all('tr')[-1].text

'\nTotal Number of Mines Found:\xa0\xa019'

## Actually scraping

### Hopefully you know that each `tr` is supposed to be your data. What is the index of the first row element that is actually a result?

`.text` will help you here.

In [108]:
doc.find_all('tr')[7].text.strip()

'3503598\n\nOR\xa0\n Newberg Rock & Dirt \xa0\nNewberg Rock & Dirt\nSurface             \nM\xa0\nActive\xa0 \nCrushed, Broken Stone NEC'

### Loop through each operator result, printing its name

Use LIST SLICING to skip the non-data row(s).

In [142]:
tr = doc.find_all('tr')[7] # display the first tr-tag with the data we are searching for
tr

<tr>
<td align="center">
<form action="/drs/ASP/BasicMineInfostatecounty.asp" method="post" name="search">
<input name="MineId" type="hidden" value="3503598"/><font style="FONT-SIZE:.75em;">3503598</font>
</form></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --><b>OR</b><!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --> Newberg Rock &amp; Dirt<!-- /DNT -->  </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT -->Newberg Rock &amp; Dirt<!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->Surface             <!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->M<!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;">Active  </font></td>
<td><font style="FONT-SIZE:.75em;">Crushed, Broken Stone NEC  </font></td>
<th bgcolor="#000000"><input alt="More Information" border="0" name="submit" src="/drs/images/moreinfo.jpg" type="image"/></th></tr>

In [138]:
print (tr.find_all('td')[0]) # display the td-tags, finding the one with the name-data
print (tr.find_all('td')[1])
print (tr.find_all('td')[2])
print (tr.find_all('td')[3])
print (tr.find_all('td')[4])
print (tr.find_all('td')[5])
print (tr.find_all('td')[6])
print (tr.find_all('td')[7])

<td align="center">
<form action="/drs/ASP/BasicMineInfostatecounty.asp" method="post" name="search">
<input name="MineId" type="hidden" value="2103518"/><font style="FONT-SIZE:.75em;">2103518</font>
</form></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --><b>MN</b><!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT -->Vogt Dirt Service<!-- /DNT -->  </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT -->Crusher<!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->Surface             <!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->M<!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;">Intermittent  </font></td>
<td><font style="FONT-SIZE:.75em;">Construction Sand and Gravel  </font></td>


In [130]:
operator = tr.find_all('td')[2].text.strip()

In [131]:
trs = doc.find_all('tr')[7:26]
len(trs)

19

In [137]:
for tr in trs:
    print (tr.find_all('td')[2].text.strip())

Newberg Rock & Dirt
AM Dirtworks & Aggregate Sales
Dirt Company
Dirt Con
Dirt Doctor Inc
Dirt Works
Holley Dirt Company, Inc
Krueger Brothers Gravel & Dirt
M R Dirt
M.R. Dirt Inc.
P B Dirt Movers, Inc
PB Dirt Movers
PB Dirt Movers, Inc
Prescott Dirt, LLC
R D Blankenship Dirt Work LLC
SIMPSON DIRTWORX LLC
SIMPSON DIRTWORX LLC
Spry's Dirt & Gravel, Inc.
Vogt Dirt Service


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [118]:
operator_id = tr.find_all('td')[0].text.strip()

In [136]:
for tr in trs:
    print (tr.find_all('td')[0].text.strip())
    current['operator_id'] = tr.find_all('td')[0].text.strip()

3503598
4801789
5001797
4608254
2103723
4104757
0801306
3901432
3609624
3609931
1519799
4407296
4407270
0203332
2901986
4300768
4300776
2302283
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

In [143]:
scrape_mines = []
for tr in doc.find_all('tr')[7:26]:
    current = {}
    current["operator_id"] = tr.find_all('td')[0].text.strip()
    current["operator_name"] = tr.find_all('td')[2].text.strip()
    current["mine_name"] = tr.find_all('td')[3].text.strip()
    current["state"] = tr.find_all('td')[1].text.strip()
    current["mine_type"] = tr.find_all('td')[4].text.strip()
    current["coal_or_metal"] = tr.find_all('td')[5].text.strip()
    current["status"] = tr.find_all('td')[6].text.strip()
    current["commodity"] = tr.find_all('td')[7].text.strip()
    scrape_mines.append(current)

### Save that to a CSV

In [146]:
df = pd.DataFrame(scrape_mines)

In [147]:
df.to_csv("scrape_mines.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [148]:
pd.read_csv("scrape_mines.csv")

,coal_or_metal,commodity,mine_name,mine_type,operator_id,operator_name,state,status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,3503598,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,AM Dirtworks & Aggregate Sales,Surface,4801789,AM Dirtworks & Aggregate Sales,ND,Intermittent
2,M,Construction Sand and Gravel,Bush Pilot,Surface,5001797,Dirt Company,AK,Intermittent
3,M,"Crushed, Broken Limestone NEC",Hog Lick Quarry,Surface,4608254,Dirt Con,WV,Temporarily Idled
4,M,Construction Sand and Gravel,Rock Lake Plant,Surface,2103723,Dirt Doctor Inc,MN,Intermittent
5,M,Construction Sand and Gravel,Portable #1,Surface,4104757,Dirt Works,TX,Intermittent
6,M,"Sand, Common",River Road Pit,Surface,801306,"Holley Dirt Company, Inc",FL,Active
7,M,Construction Sand and Gravel,PORTABLE SCREENER,Surface,3901432,Krueger Brothers Gravel & Dirt,SD,Intermittent
8,M,Construction Sand and Gravel,Forbes Pit,Surface,3609624,M R Dirt,PA,Intermittent
9,M,Dimension Stone NEC,Camptown Quarry,Surface,3609931,M.R. Dirt Inc.,PA,Intermittent
